In [150]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import string
import itertools
import datetime

In [3]:
url = 'https://www.oddschecker.com/'

In [4]:
country_code = ['UK','IRE','USA','AUS']

In [5]:
def get_soup(base_url, sport = 'horses', event_url = None):
    '''Uses beautiful soup to get parse the url
    base_url = str, www.oddschecker.com/
    sport = str, which sport do you want to look at
    event_url = str, of the url extension which will take you to the '''
    
    if sport == 'horses':
        sport = 'horse-racing'
    
    url = base_url + sport
    if event_details != None:
        url += event_url
    
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    return soup(webpage, "html.parser")
    

In [6]:
page_soup = get_soup(url)


https://www.oddschecker.com/horse-racing


In [7]:
print(type(page_soup))

<class 'bs4.BeautifulSoup'>


In [141]:
def get_races(bsoup, country_codes, sport = 'horses'):
    '''Will return a dictionary of the events displayed on www.oddschecker.com
        Only does horse_racing atm.
        dict structure = events[countrycode][venue][list of event times]
        bsoup = the page parse with beautifulsoup4
        country_codes = countries you want to get events for
        sport = the sport you want''' 
    events = {code:{} for code in country_codes}
    containers = bsoup.findAll('div', {'class' : 'race-details'})
    for container in containers:
        txt = container.find('div', {'class' : 'venue-details'}).text
        
        for code in country_codes:
            # get country code and venue
            if code in txt[:3]:
                cc = code
                venue = txt.replace(code, '')
                break

        # get event times   
        times = [x for x in container.findAll('div', {'class' : 'racing-time'})]
        events[cc][venue] = times
        
    return events


In [142]:
events = get_races(page_soup, country_code)

In [145]:
class race():
    def __init__(self,base_url, sport, cc, venue, time):
        '''have a race as a class which we can add horse classes to.'''
        self.cc = cc
        self.venue = venue,
        self.time = time
        self.url_ext = '/' + self.venue + '/' + self.time + '/' + 'winner'
        
        # soup the url
        soup = get_soup(base_url, sport, event_url = self.url_ext)
        
        # get list of horses
        
        #__init__ the horse class
        
        # get bookies
            # I assume this is the same for all races but maybe not.
        
    def __str__(self):
        return f'{self.venue}, {self.cc} at {self.time}'
        
    def get_race_date(self,bsoup):
        '''placeholder for method to get the date of the race.  
        Will either be today or tomorrow on the website but haven''t worked it out how to grab it yet
        might work better in get races function'''
        pass
        
        

In [ ]:
def horse():
    '''placeholder for a horse class'''
    
    
    def __init__(self,name, soup):
        '''will just be a name with '''
        pass
    def __str__(self,soup):
        pass
    